# 사전 준비

## 1. 패키지 임포트

In [2]:
import pandas
import sqlite3

## 2. 예제 데이터 준비

In [13]:
t1 = pandas.DataFrame([
    { 'c1': 11, 'c2': 111, 'c3': 1111, },
    { 'c1': 12, 'c2': 121, 'c3': 1211, },
    { 'c1': 13, 'c2': 131, 'c3': 1311, },
    { 'c1': 14, 'c2': 141, 'c3': 1411, },
    { 'c1': 15, 'c2': 151, 'c3': 1511, },
    { 'c1': 16, 'c2': 161, 'c3': 1611, },
])
t2 = pandas.DataFrame([
    { 'c1': 11, 'c2': 111, 'c3': 1111, 'c4': 21111, 'c5': 211111, },
    { 'c1': 12, 'c2': 121, 'c3': 1211, 'c4': 22111, 'c5': 221111, },
    { 'c1': 13, 'c2': 131, 'c3': 1311, 'c4': 23111, 'c5': 231111, },
    { 'c1': 21, 'c2': 211, 'c3': 2111, 'c4': 21111, 'c5': 211111, },
    { 'c1': 22, 'c2': 221, 'c3': 2211, 'c4': 22111, 'c5': 221111, },
    { 'c1': 23, 'c2': 231, 'c3': 2311, 'c4': 23111, 'c5': 231111, },
    { 'c1': 24, 'c2': 241, 'c3': 2411, 'c4': 24111, 'c5': 241111, },
    { 'c1': 25, 'c2': 251, 'c3': 2511, 'c4': 25111, 'c5': 251111, },
    { 'c1': 26, 'c2': 261, 'c3': 2611, 'c4': 26111, 'c5': 261111, },
])

db_conn = sqlite3.connect(':memory:')
t1.to_sql('t1', db_conn, if_exists='replace')
t2.to_sql('t2', db_conn, if_exists='replace')

# UNION

## 1. Python

In [15]:
df = pandas.concat([t1, t2], sort=False)
df = df[['c1', 'c2', 'c3']]
df = df.drop_duplicates()
print(df.shape)
display(df)

(12, 3)


,c1,c2,c3
0,11,111,1111
1,12,121,1211
2,13,131,1311
3,14,141,1411
4,15,151,1511
5,16,161,1611
3,21,211,2111
4,22,221,2211
5,23,231,2311
6,24,241,2411


## 2. SQL

In [17]:
sql_select = '''
select c1, c2, c3
  from t1
 union
select c1, c2, c3
  from t2
'''
df_from_db = pandas.read_sql(sql_select, db_conn)
print(df_from_db.shape)
display(df_from_db)

(12, 3)


,c1,c2,c3
0,11,111,1111
1,12,121,1211
2,13,131,1311
3,14,141,1411
4,15,151,1511
5,16,161,1611
6,21,211,2111
7,22,221,2211
8,23,231,2311
9,24,241,2411


# UNION ALL

## 1. Python

In [18]:
df = pandas.concat([t1, t2], sort=False)
df = df[['c1', 'c2', 'c3']]
print(df.shape)
display(df)

(15, 3)


,c1,c2,c3
0,11,111,1111
1,12,121,1211
2,13,131,1311
3,14,141,1411
4,15,151,1511
5,16,161,1611
0,11,111,1111
1,12,121,1211
2,13,131,1311
3,21,211,2111


## 2. SQL

In [19]:
sql_select = '''
select c1, c2, c3
  from t1
 union all
select c1, c2, c3
  from t2
'''
df_from_db = pandas.read_sql(sql_select, db_conn)
print(df_from_db.shape)
display(df_from_db)

(15, 3)


,c1,c2,c3
0,11,111,1111
1,12,121,1211
2,13,131,1311
3,14,141,1411
4,15,151,1511
5,16,161,1611
6,11,111,1111
7,12,121,1211
8,13,131,1311
9,21,211,2111


# RELATIVE COMPLEMENT

## 1. Python

In [21]:
set_t2 = set(t2['c1'].tolist())
df = t1[~t1['c1'].isin(set_t2)]
print(df.shape)
display(df)

(3, 3)


,c1,c2,c3
3,14,141,1411
4,15,151,1511
5,16,161,1611


## 2. SQL

In [22]:
sql_select = '''
select c1, c2, c3
  from t1
 where not exists (select 1
                     from t2
                    where t1.c1 = t2.c1)
'''
df_from_db = pandas.read_sql(sql_select, db_conn)
print(df_from_db.shape)
display(df_from_db)

(3, 3)


,c1,c2,c3
0,14,141,1411
1,15,151,1511
2,16,161,1611


# INNER JOIN

## 1. Python

In [30]:
df_t1 = t1.set_index(['c1'])
df_t2 = t2.set_index(['c1'])
df = df_t1.join(df_t2, how='inner', lsuffix='', rsuffix='_DEL')
df = df.reset_index()
df = df[['c1', 'c2', 'c3', 'c4']]
print(df.shape)
display(df)

(3, 4)


,c1,c2,c3,c4
0,11,111,1111,21111
1,12,121,1211,22111
2,13,131,1311,23111


## 2. SQL

In [31]:
sql_select = '''
select t1.c1, t1.c2, t1.c3, t2.c4
  from t1
 inner join t2
         on t1.c1 = t2.c1
'''
df_from_db = pandas.read_sql(sql_select, db_conn)
print(df_from_db.shape)
display(df_from_db)

(3, 4)


,c1,c2,c3,c4
0,11,111,1111,21111
1,12,121,1211,22111
2,13,131,1311,23111


# LEFT OUTER JOIN

## 1. Python

In [34]:
df_t1 = t1.set_index(['c1'])
df_t2 = t2.set_index(['c1'])
df = df_t1.join(df_t2, how='left', lsuffix='', rsuffix='_DEL')
df = df.reset_index()
df = df[['c1', 'c2', 'c3', 'c4']]
print(df.shape)
display(df)

(6, 4)


,c1,c2,c3,c4
0,11,111,1111,21111.0
1,12,121,1211,22111.0
2,13,131,1311,23111.0
3,14,141,1411,NaN
4,15,151,1511,NaN
5,16,161,1611,NaN


## 2. SQL

In [35]:
sql_select = '''
select t1.c1, t1.c2, t1.c3, t2.c4
  from t1
  left outer join t2
               on t1.c1 = t2.c1
'''
df_from_db = pandas.read_sql(sql_select, db_conn)
print(df_from_db.shape)
display(df_from_db)

(6, 4)


,c1,c2,c3,c4
0,11,111,1111,21111.0
1,12,121,1211,22111.0
2,13,131,1311,23111.0
3,14,141,1411,NaN
4,15,151,1511,NaN
5,16,161,1611,NaN


# GROUP BY

## 1. Python

In [39]:
df = t1.groupby(['c1', 'c2']).agg( cnt_c3 = ( 'c3', 'count')).reset_index()
print(df.shape)
display(df)

(6, 3)


,c1,c2,cnt_c3
0,11,111,1
1,12,121,1
2,13,131,1
3,14,141,1
4,15,151,1
5,16,161,1


## 2. SQL

In [40]:
sql_select = '''
select c1, c2, count(c3) as cnt_c3
  from t1
 group by c1, c2
'''
df_from_db = pandas.read_sql(sql_select, db_conn)
print(df_from_db.shape)
display(df_from_db)

(6, 3)


,c1,c2,cnt_c3
0,11,111,1
1,12,121,1
2,13,131,1
3,14,141,1
4,15,151,1
5,16,161,1


# WHERE

## 1. Python

In [41]:
df = t2[t2['c4'] == 22111]
df = df[['c1', 'c2', 'c3']]
print(df.shape)
display(df)

(2, 3)


,c1,c2,c3
1,12,121,1211
4,22,221,2211


## 2. SQL

In [42]:
sql_select = '''
select c1, c2, c3
  from t2
 where c4 = 22111
'''
df_from_db = pandas.read_sql(sql_select, db_conn)
print(df_from_db.shape)
display(df_from_db)

(2, 3)


,c1,c2,c3
0,12,121,1211
1,22,221,2211


# HAVING

## 1. Python

In [45]:
import numpy
df = t1.groupby(['c1', 'c2']).agg( cnt_c3 = ( 'c3', numpy.sum)).reset_index()
df = df[df['cnt_c3'] > 1311]
print(df.shape)
display(df)

(3, 3)


,c1,c2,cnt_c3
3,14,141,1411
4,15,151,1511
5,16,161,1611


In [48]:
sql_select = '''
select c1, c2, sum(c3) as cnt_c3
  from t1
 group by c1, c2
having sum(c3) > 1311
'''
df_from_db = pandas.read_sql(sql_select, db_conn)
print(df_from_db.shape)
display(df_from_db)

(3, 3)


,c1,c2,cnt_c3
0,14,141,1411
1,15,151,1511
2,16,161,1611
